# Blackjack with AI Player

Insert introduction discussion

In [212]:
import random

class Blackjack:
    
    MOVES = ['Hit', 'Double Down', 'Split', 'Stand', 'Surrender']
    
    def __init__(self):
        self.deck = Deck()
        self.player = Player()
    
    def hit(self):
        return self.deck.draw()
        

class Player:
    
    def __init__(self):
        self.money = 5000
        self.bet = 0
        self.hands = [Hand()]
        
    
    def doubleDown(self):
        self.bet *= 2
    
    def split(self, hand):
        newHand = Hand(self.hands.index(hand).removeCard())
        self.hands.append(newHand)
        
    def surrender(self):
        return
    
    # bet a flat $50 
    def betMoney(self):
        self.bet = 50
        
    def showHands(self):
        print("Player has %d hands: " % len(self.hands))
        
        for hand in self.hands:
            #print(hand)
            return
        
        
class Hand:
    
    def __init__(self, card = None):
        if card:
            self.hand = [card]
        else:
            self.hand = []
        
    def addCard(self, card):
        self.hand.append(card)
    
    def removeCard(self):
        return self.hand.pop()
    
    def __str__(self):
        return str([card.name for card in self.hand])
    
    
class Deck:
    
    def __init__(self):
        self.deck = self.buildDeck()
        self.prepareDeck()
        
    def buildDeck(self):
        deck = []
        
        for i in range(4):
            for j in range(len(Card.CARDS)):
                deck.append(Card(Card.CARDS[j]))
                
        return deck
    
    # shuffle the deck 7 times to prepare it to be used - 7 times is the researched number 
    # of times necassary to consider a deck of 52 cards as properly shuffled
    def prepareDeck(self):
        for i in range(7):
            self.shuffle()
    
    def shuffle(self):
        random.shuffle(self.deck)      
    def draw(self):
        card = self.deck.pop(0)
        return card
    def __len__(self):
        return len(self.deck)
    
    def __str__(self):
        return str([card.name for card in self.deck])
        
class Card:
    
    FACES = {'A': 1, 'J': 10, 'Q': 10, 'K': 10}
    CARDS = ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']
    
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return "{0.name}".format(self)
    
    @property 
    def value(self):
        return self.FACES.get(self.name, self.name)

In [215]:
game = Blackjack()
print(len(game.deck))
print(game.deck)
print(game.hit())
print(len(game.deck))

game.player.hands[0].addCard(game.hit())
print(game.player.showHands())

52
['J', 'A', 'Q', 'J', '9', 'K', '3', '10', '10', '10', 'K', '6', '5', 'K', '7', '8', '9', '2', '7', '6', '9', 'Q', '10', '6', '4', '2', '7', '5', '2', '4', '3', 'K', '4', 'J', '8', 'Q', '3', 'Q', 'A', '5', '6', '3', '5', 'A', '8', 'A', '2', '9', '4', '7', 'J', '8']
J
51
Player has 1 hands: 
None
